# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [1]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [2]:
# Standard library imports
import os
import json

# ChromaDB for vector database access
import chromadb
from chromadb.utils import embedding_functions

# Environment variables
from dotenv import load_dotenv

# Pydantic for structured outputs
from pydantic import BaseModel, Field

# Tavily for web search
from tavily import TavilyClient

# Our custom library imports (from lib/ folder)
from lib.agents import Agent
from lib.llm import LLM
from lib.messages import UserMessage, SystemMessage, ToolMessage, AIMessage
from lib.tooling import tool

print("All imports loaded successfully!")

All imports loaded successfully!


In [3]:
# Load environment variables from .env file
load_dotenv("../../.env")

# Get API keys
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
CHROMA_OPENAI_API_KEY = os.getenv("CHROMA_OPENAI_API_KEY", OPENAI_API_KEY)
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

# Verify keys are loaded
print(f"OpenAI API Key loaded: {'Yes' if OPENAI_API_KEY else 'No'}")
print(f"Tavily API Key loaded: {'Yes' if TAVILY_API_KEY else 'No'}")

OpenAI API Key loaded: Yes
Tavily API Key loaded: Yes


In [4]:
# Connect to our existing ChromaDB (created in Part 1)
chroma_client = chromadb.PersistentClient(path="chromadb")

# Get the collection with the same embedding function we used to create it
embedding_fn = embedding_functions.OpenAIEmbeddingFunction(
    api_key=CHROMA_OPENAI_API_KEY,
    model_name="text-embedding-3-small"
)
collection = chroma_client.get_collection(name="udaplay", embedding_function=embedding_fn)

# Initialize Tavily client for web search
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

print(f"ChromaDB collection 'udaplay' loaded with {collection.count()} documents")
print(f"Tavily client initialized")

ChromaDB collection 'udaplay' loaded with 15 documents
Tavily client initialized


### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [5]:
@tool
def retrieve_game(query: str) -> str:
    """
    Semantic search: Finds most relevant games in the vector database.
    
    Args:
        query: A question about the game industry
    
    Returns results as a list where each element contains:
    - Platform: like Game Boy, PlayStation 5, Xbox 360...
    - Name: Name of the Game
    - YearOfRelease: Year when that game was released for that platform
    - Description: Additional details about the game
    """
    # Query the vector database
    results = collection.query(
        query_texts=[query],
        n_results=3,
        include=["metadatas", "documents", "distances"]
    )
    
    # Format results as a readable string for the LLM
    if not results["metadatas"][0]:
        return "No games found matching the query."
    
    formatted_results = []
    for i, metadata in enumerate(results["metadatas"][0]):
        game_info = (
            f"Game {i+1}:\n"
            f"  Name: {metadata['Name']}\n"
            f"  Platform: {metadata['Platform']}\n"
            f"  Year of Release: {metadata['YearOfRelease']}\n"
            f"  Genre: {metadata['Genre']}\n"
            f"  Publisher: {metadata['Publisher']}\n"
            f"  Description: {metadata['Description']}"
        )
        formatted_results.append(game_info)
    
    return "\n\n".join(formatted_results)

# Test the tool
print("Testing retrieve_game tool:")
print(retrieve_game("Mario platformer games"))

Testing retrieve_game tool:
Game 1:
  Name: Super Mario World
  Platform: Super Nintendo Entertainment System (SNES)
  Year of Release: 1990
  Genre: Platformer
  Publisher: Nintendo
  Description: A classic platformer where Mario embarks on a quest to save Princess Toadstool and Dinosaur Land from Bowser.

Game 2:
  Name: Super Mario 64
  Platform: Nintendo 64
  Year of Release: 1996
  Genre: Platformer
  Publisher: Nintendo
  Description: A groundbreaking 3D platformer that set new standards for the genre, featuring Mario's quest to rescue Princess Peach.

Game 3:
  Name: Super Smash Bros. Melee
  Platform: GameCube
  Year of Release: 2001
  Genre: Fighting
  Publisher: Nintendo
  Description: A crossover fighting game featuring characters from various Nintendo franchises battling it out in dynamic arenas.


#### Evaluate Retrieval Tool

In [6]:
# Define structured output for evaluation results
class EvaluationReport(BaseModel):
    """Structured response from the evaluation LLM"""
    useful: bool = Field(description="Whether the documents are useful to answer the question")
    explanation: str = Field(description="Detailed explanation of the evaluation result")


@tool
def evaluate_retrieval(question: str, retrieved_docs: str) -> str:
    """
    Evaluates if the retrieved documents can answer the user's question.
    
    Uses an LLM as a judge to analyze whether the retrieved documents
    contain enough information to properly respond to the question.
    
    Args:
        question: The original question from the user
        retrieved_docs: The documents retrieved from the vector database
    
    Returns:
        A JSON string with:
        - useful: whether the documents are useful to answer the question
        - explanation: description about the evaluation result
    """
    # Create an LLM instance for evaluation
    judge_llm = LLM(
        model="gpt-4o-mini",
        temperature=0.0  # We want consistent, deterministic judgments
    )
    
    # Create the evaluation prompt
    eval_prompt = f"""Your task is to evaluate if the retrieved documents contain enough information to answer the user's question.

USER'S QUESTION:
{question}

RETRIEVED DOCUMENTS:
{retrieved_docs}

Analyze carefully:
1. Do the documents contain information directly relevant to the question?
2. Can the question be fully answered using ONLY the information in these documents?
3. Is there any missing information that would be needed?

Respond with your evaluation."""

    # Get structured response from LLM
    response = judge_llm.invoke(
        input=eval_prompt,
        response_format=EvaluationReport
    )
    
    # Parse the response into our structured format
    try:
        evaluation = EvaluationReport.model_validate_json(response.content)
        return json.dumps({
            "useful": evaluation.useful,
            "explanation": evaluation.explanation
        })
    except Exception as e:
        # Fallback if parsing fails
        return json.dumps({
            "useful": False,
            "explanation": f"Evaluation failed: {str(e)}"
        })


# Test the tool
print("Testing evaluate_retrieval tool:")
test_docs = retrieve_game("Pokemon Gold and Silver release date")
result = evaluate_retrieval("When was Pokemon Gold and Silver released?", test_docs)
print(result)

Testing evaluate_retrieval tool:
{"useful": true, "explanation": "The retrieved documents contain relevant information about the release date of Pok\u00e9mon Gold and Silver, specifically stating that it was released in 1999. This directly answers the user's question. The other documents do not pertain to the question and are not necessary for answering it. Therefore, the question can be fully answered using the information provided in the first document."}


#### Game Web Search Tool

In [7]:
@tool
def game_web_search(question: str) -> str:
    """
    Search the web for video game information when internal knowledge is insufficient.
    
    Use this tool as a fallback when the vector database doesn't have
    enough information to answer the user's question.
    
    Args:
        question: A question about the game industry
    
    Returns:
        Web search results with relevant information from the internet.
    """
    # Search the web using Tavily
    response = tavily_client.search(
        query=question,
        max_results=3,
        search_depth="basic"  # Use "advanced" for more thorough but slower search
    )
    
    # Format results for the LLM
    if not response.get("results"):
        return "No web results found for this query."
    
    formatted_results = []
    for i, result in enumerate(response["results"]):
        result_info = (
            f"Source {i+1}:\n"
            f"  Title: {result.get('title', 'N/A')}\n"
            f"  URL: {result.get('url', 'N/A')}\n"
            f"  Content: {result.get('content', 'N/A')}"
        )
        formatted_results.append(result_info)
    
    return "\n\n".join(formatted_results)


# Test the tool
print("Testing game_web_search tool:")
print(game_web_search("What are the best video games of 2024?"))

Testing game_web_search tool:
Source 1:
  Title: Ars Technica's top 20 video games of 2024
  URL: https://arstechnica.com/gaming/2024/12/ars-technicas-top-20-video-games-of-2024/
  Content: Ars Technica's top 20 video games of 2024 · Animal Well · Astro Bot · Balatro · The Crimson Diamond · Elden Ring: Shadow of the Erdtree.

Source 2:
  Title: Blogcritics' Top 10 Videogames of 2024
  URL: https://blogcritics.org/blogcritics-top-10-videogames-of-2024/
  Content: Blogcritics' Top 10 Videogames of 2024 · No. 10 – 1000xRESIST · No. 9: Dragon Age: The Veilguard · No. 8: Stellar Blade · No. 7: The Plucky

Source 3:
  Title: The Best Games of 2024 Ranked
  URL: https://www.supergamesite64.com/opinion/the-best-games-of-2024-ranked/
  Content: 10. Dragon's Dogma II · 9. Tekken 8 · 8. Kunitsu-Gami: Path of the Goddess · 7. Prince of Persia: The Lost Crown · 6. Warhammer 40,0000: Space


### Agent

In [8]:
# Define the agent's instructions
# These guide the agent's behavior and tool usage
AGENT_INSTRUCTIONS = """You are UdaPlay, an AI Research Agent specialized in the video game industry.

Your goal is to answer questions about video games accurately using available tools.

## Workflow (follow this order):

1. **FIRST**: Use `retrieve_game` to search the internal database for relevant information.

2. **THEN**: Use `evaluate_retrieval` to assess if the retrieved documents can answer the question.
   - Pass the original question AND the retrieved documents to this tool.

3. **DECISION**:
   - If evaluation says "useful: true" → Use the retrieved information to answer.
   - If evaluation says "useful: false" → Use `game_web_search` to find information online.

4. **ANSWER**: Provide a clear, well-structured answer with:
   - Direct answer to the question
   - Supporting details
   - Citation of source (either "Source: Internal Database" or the web URL)

## Important Rules:
- Always follow the workflow: retrieve → evaluate → answer (or web search → answer)
- Never make up information - only use data from tools
- If you cannot find the answer anywhere, say so honestly
- Keep answers concise but complete
"""

# Create the agent with all tools
agent = Agent(
    model_name="gpt-4o-mini",
    instructions=AGENT_INSTRUCTIONS,
    tools=[retrieve_game, evaluate_retrieval, game_web_search],
    temperature=0.7
)

print("UdaPlay Agent created successfully!")
print(f"Model: gpt-4o-mini")
print(f"Tools: {[t.name for t in agent.tools]}")

UdaPlay Agent created successfully!
Model: gpt-4o-mini
Tools: ['retrieve_game', 'evaluate_retrieval', 'game_web_search']


In [9]:
def run_query_with_report(query: str, session_id: str = None):
    """
    Run a query through the agent and display a detailed report.
    
    Shows:
    - The original query
    - Tool calls made (reasoning)
    - Tool results
    - Final answer with citations
    """
    print("=" * 70)
    print(f"QUERY: {query}")
    print("=" * 70)
    
    # Run the agent
    run = agent.invoke(query, session_id=session_id)
    
    # Get the final state
    final_state = run.get_final_state()
    messages = final_state.get("messages", [])
    
    # Track tool usage
    tool_calls_made = []
    
    print("\n📋 AGENT REASONING & TOOL USAGE:")
    print("-" * 50)
    
    for msg in messages:
        # Show AI decisions and tool calls
        if isinstance(msg, AIMessage):
            if msg.tool_calls:
                for tc in msg.tool_calls:
                    tool_name = tc.function.name
                    tool_args = json.loads(tc.function.arguments)
                    tool_calls_made.append(tool_name)
                    print(f"\n🔧 Tool Called: {tool_name}")
                    # Show truncated args for readability
                    for key, value in tool_args.items():
                        display_value = str(value)[:100] + "..." if len(str(value)) > 100 else value
                        print(f"   └─ {key}: {display_value}")
        
        # Show tool results (truncated)
        elif isinstance(msg, ToolMessage):
            result_preview = msg.content[:200] + "..." if len(msg.content) > 200 else msg.content
            print(f"\n📄 Tool Result ({msg.name}):")
            print(f"   {result_preview}")
    
    # Show final answer
    print("\n" + "=" * 70)
    print("💬 FINAL ANSWER:")
    print("-" * 50)
    
    # Get the last AI message with content (the final answer)
    final_answer = None
    for msg in reversed(messages):
        if isinstance(msg, AIMessage) and msg.content and not msg.tool_calls:
            final_answer = msg.content
            break
    
    if final_answer:
        print(final_answer)
    else:
        print("No final answer generated.")
    
    print("\n" + "=" * 70)
    print(f"📊 SUMMARY: Tools used: {tool_calls_made}")
    print(f"   Total tokens: {final_state.get('total_tokens', 'N/A')}")
    print("=" * 70 + "\n\n")
    
    return run

#### Example Query 1: Pokemon Gold and Silver Release Date
This should be answerable from our internal database.

In [10]:
# Query 1: Should find answer in internal database
run1 = run_query_with_report("When was Pokémon Gold and Silver released?")

QUERY: When was Pokémon Gold and Silver released?
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

📋 AGENT REASONING & TOOL USAGE:
--------------------------------------------------

🔧 Tool Called: retrieve_game
   └─ query: Pokémon Gold and Silver release date

📄 Tool Result (retrieve_game):
   "Game 1:\n  Name: Pok\u00e9mon Gold and Silver\n  Platform: Game Boy Color\n  Year of Release: 1999\n  Genre: Role-playing\n  Publisher: Nintendo\n  Description: Second-generation Pok\u00e9mon games i...

🔧 Tool Called: evaluate_retrieval
   └─ question: When was Pokémon Gold and Silver released?
   └─ retrieved_docs: Game 1:
  Name: Pok mon Gold and Silver
  Platform: Game Boy Color
  Year of Releas

#### Example Query 2: First 3D Mario Platformer
This should also be answerable from our internal database.

In [11]:
# Query 2: Should find answer in internal database
run2 = run_query_with_report("Which one was the first 3D platformer Mario game?")

QUERY: Which one was the first 3D platformer Mario game?
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

📋 AGENT REASONING & TOOL USAGE:
--------------------------------------------------

🔧 Tool Called: retrieve_game
   └─ query: Pokémon Gold and Silver release date

📄 Tool Result (retrieve_game):
   "Game 1:\n  Name: Pok\u00e9mon Gold and Silver\n  Platform: Game Boy Color\n  Year of Release: 1999\n  Genre: Role-playing\n  Publisher: Nintendo\n  Description: Second-generation Pok\u00e9mon games i...

🔧 Tool Called: evaluate_retrieval
   └─ question: When was Pokémon Gold and Silver released?
   └─ retrieved_docs: Game 1:
  Name: Pok mon Gold and Silver
  Platform: Game Boy Color
  Year of

#### Example Query 3: Mortal Kombat X on PS5
This may require web search if our database doesn't have this information.

In [12]:
# Query 3: May need web search fallback
run3 = run_query_with_report("Was Mortal Kombat X released for PlayStation 5?")

QUERY: Was Mortal Kombat X released for PlayStation 5?
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

📋 AGENT REASONING & TOOL USAGE:
--------------------------------------------------

🔧 Tool Called: retrieve_game
   └─ query: Pokémon Gold and Silver release date

📄 Tool Result (retrieve_game):
   "Game 1:\n  Name: Pok\u00e9mon Gold and Silver\n  Platform: Game Boy Color\n  Year of Release: 1999\n  Genre: Role-playing\n  Publisher: Nintendo\n  Description: Second-generation Pok\u00e9mon games i...

🔧 Tool Called: evaluate_retrieval
   └─ question: When was Pokémon Gold and Silver released?
   └─ retr

### Summary

In this notebook, we successfully built the **UdaPlay AI Agent** with:

1. **Three Tools:**
   - `retrieve_game` - Searches the vector database for game information
   - `evaluate_retrieval` - Uses LLM-as-judge to assess if retrieved docs can answer the question
   - `game_web_search` - Falls back to Tavily web search when internal knowledge is insufficient

2. **Agent Workflow:**
   - First attempts to answer using internal knowledge (RAG)
   - Evaluates the quality of retrieved results
   - Falls back to web search if needed
   - Provides clear answers with citations

3. **State Machine Architecture:**
   - Maintains conversation state across queries
   - Tracks tool usage and execution flow
   - Provides detailed reporting of agent reasoning

### (Optional) Advanced Feature 1: Structured Output

Return answers in both natural language AND structured JSON format for easy integration with other systems.

In [13]:
# Define structured output model for agent responses
class GameAnswer(BaseModel):
    """Structured format for agent responses"""
    answer: str = Field(description="The natural language answer to the question")
    sources: list = Field(description="List of sources used (database or URLs)")
    confidence: str = Field(description="Confidence level: high, medium, or low")
    tools_used: list = Field(description="List of tools that were called")


def get_structured_response(run) -> dict:
    """
    Extract a structured response from an agent run.
    
    Returns both natural language and JSON formatted output.
    """
    final_state = run.get_final_state()
    messages = final_state.get("messages", [])
    
    # Extract the final answer
    final_answer = None
    for msg in reversed(messages):
        if isinstance(msg, AIMessage) and msg.content and not msg.tool_calls:
            final_answer = msg.content
            break
    
    # Extract tools used and sources
    tools_used = []
    sources = []
    used_web_search = False
    
    for msg in messages:
        if isinstance(msg, AIMessage) and msg.tool_calls:
            for tc in msg.tool_calls:
                tool_name = tc.function.name
                if tool_name not in tools_used:
                    tools_used.append(tool_name)
                if tool_name == "game_web_search":
                    used_web_search = True
        
        # Extract URLs from web search results
        if isinstance(msg, ToolMessage) and msg.name == "game_web_search":
            content = msg.content
            # Parse URLs from the content
            import re
            urls = re.findall(r'URL: (https?://[^\s]+)', content)
            sources.extend(urls)
    
    # Determine sources
    if not used_web_search:
        sources = ["Internal Database (ChromaDB)"]
    elif not sources:
        sources = ["Web Search"]
    
    # Determine confidence based on workflow
    if "retrieve_game" in tools_used and not used_web_search:
        confidence = "high"  # Found in our database
    elif used_web_search:
        confidence = "medium"  # Had to search the web
    else:
        confidence = "low"
    
    # Create structured response
    structured = GameAnswer(
        answer=final_answer or "No answer generated",
        sources=sources,
        confidence=confidence,
        tools_used=tools_used
    )
    
    return {
        "natural_language": final_answer,
        "structured_json": structured.model_dump()
    }


# Test with a new query
print("Testing Structured Output:")
print("=" * 60)

test_run = agent.invoke("What year was Gran Turismo released?", session_id="structured_test")
result = get_structured_response(test_run)

print("\n📝 NATURAL LANGUAGE:")
print(result["natural_language"])

print("\n📊 STRUCTURED JSON:")
print(json.dumps(result["structured_json"], indent=2))

Testing Structured Output:
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

📝 NATURAL LANGUAGE:
Gran Turismo was released in 1997 for the PlayStation 1. It is known as a realistic racing simulator that set a new standard for the genre. 

Source: Internal Database

📊 STRUCTURED JSON:
{
  "answer": "Gran Turismo was released in 1997 for the PlayStation 1. It is known as a realistic racing simulator that set a new standard for the genre. \n\nSource: Internal Database",
  "sources": [
    "Internal Database (ChromaDB)"
  ],
  "confidence": "high",
  "tools_used": [
    "retrieve_game",
    "evaluate_retrieval"
  ]
}


### (Optional) Advanced Feature 2: Long-term Memory

The agent can now "learn" from web searches and remember useful information for future queries. This makes the agent smarter over time!

In [14]:
# Import long-term memory classes from our library
from lib.memory import LongTermMemory, MemoryFragment
from lib.vector_db import VectorStoreManager

# Initialize the VectorStoreManager and LongTermMemory
memory_manager = VectorStoreManager(openai_api_key=OPENAI_API_KEY)
long_term_memory = LongTermMemory(db=memory_manager)

print("Long-term memory initialized!")

Long-term memory initialized!


In [15]:
@tool
def save_to_memory(information: str, topic: str) -> str:
    """
    Save important information to long-term memory for future use.
    
    Use this tool to store useful facts learned from web searches
    so they can be recalled in future conversations.
    
    Args:
        information: The factual information to remember (be specific and detailed)
        topic: A short topic/category for this information (e.g., "Pokemon release dates")
    
    Returns:
        Confirmation that the information was saved.
    """
    # Create a memory fragment
    memory = MemoryFragment(
        content=f"[{topic}] {information}",
        owner="udaplay_agent",
        namespace="game_knowledge"
    )
    
    # Save to long-term memory
    long_term_memory.register(memory, metadata={"topic": topic})
    
    return f"Successfully saved to memory: '{topic}'"


@tool  
def recall_memory(query: str) -> str:
    """
    Search long-term memory for previously learned information.
    
    Use this tool to check if we've already learned something from
    previous web searches before searching the web again.
    
    Args:
        query: What information to search for in memory
    
    Returns:
        Previously learned information if found, or indication that nothing was found.
    """
    # Search long-term memory
    results = long_term_memory.search(
        query_text=query,
        owner="udaplay_agent",
        namespace="game_knowledge",
        limit=3
    )
    
    if not results.fragments:
        return "No relevant information found in memory."
    
    # Format results
    memories = []
    for i, fragment in enumerate(results.fragments):
        memories.append(f"Memory {i+1}: {fragment.content}")
    
    return "\n".join(memories)


# Test the memory tools
print("Testing memory tools:")
print("-" * 40)

# Save something
save_result = save_to_memory(
    information="The Legend of Zelda: Tears of the Kingdom was released on May 12, 2023 for Nintendo Switch.",
    topic="Zelda release dates"
)
print(f"Save: {save_result}")

# Recall it
recall_result = recall_memory("When was Tears of the Kingdom released?")
print(f"Recall: {recall_result}")

Testing memory tools:
----------------------------------------
Save: Successfully saved to memory: 'Zelda release dates'
Recall: Memory 1: [Zelda release dates] The Legend of Zelda: Tears of the Kingdom was released on May 12, 2023 for Nintendo Switch.


In [16]:
# Create an enhanced agent with memory capabilities
ENHANCED_AGENT_INSTRUCTIONS = """You are UdaPlay, an AI Research Agent specialized in the video game industry.

Your goal is to answer questions about video games accurately using available tools.

## Workflow (follow this order):

1. **FIRST**: Use `retrieve_game` to search the internal database for relevant information.

2. **ALSO CHECK**: Use `recall_memory` to check if you've learned anything relevant from previous searches.

3. **THEN**: Use `evaluate_retrieval` to assess if the retrieved documents can answer the question.
   - Pass the original question AND the retrieved documents to this tool.

4. **DECISION**:
   - If evaluation says "useful: true" → Use the retrieved information to answer.
   - If evaluation says "useful: false" → Use `game_web_search` to find information online.

5. **LEARN**: After a successful web search, use `save_to_memory` to store the key facts you learned.
   - This helps you answer similar questions faster in the future!

6. **ANSWER**: Provide a clear, well-structured answer with:
   - Direct answer to the question
   - Supporting details
   - Citation of source (Internal Database, Memory, or web URL)

## Important Rules:
- Always follow the workflow: retrieve → recall → evaluate → answer (or web search → save → answer)
- Never make up information - only use data from tools
- Save useful web findings to memory for future use
- If you cannot find the answer anywhere, say so honestly
"""

# Create the enhanced agent with all tools including memory
enhanced_agent = Agent(
    model_name="gpt-4o-mini",
    instructions=ENHANCED_AGENT_INSTRUCTIONS,
    tools=[retrieve_game, evaluate_retrieval, game_web_search, save_to_memory, recall_memory],
    temperature=0.7
)

print("Enhanced UdaPlay Agent with Long-term Memory created!")
print(f"Model: gpt-4o-mini")
print(f"Tools: {[t.name for t in enhanced_agent.tools]}")

Enhanced UdaPlay Agent with Long-term Memory created!
Model: gpt-4o-mini
Tools: ['retrieve_game', 'evaluate_retrieval', 'game_web_search', 'save_to_memory', 'recall_memory']


In [17]:
def run_enhanced_query(query: str, session_id: str = None):
    """Run a query with the enhanced agent and show memory usage."""
    print("=" * 70)
    print(f"QUERY: {query}")
    print("=" * 70)
    
    # Run the enhanced agent
    run = enhanced_agent.invoke(query, session_id=session_id)
    
    # Get the final state
    final_state = run.get_final_state()
    messages = final_state.get("messages", [])
    
    # Track tool usage
    tool_calls_made = []
    
    print("\n📋 AGENT REASONING & TOOL USAGE:")
    print("-" * 50)
    
    for msg in messages:
        if isinstance(msg, AIMessage):
            if msg.tool_calls:
                for tc in msg.tool_calls:
                    tool_name = tc.function.name
                    tool_args = json.loads(tc.function.arguments)
                    tool_calls_made.append(tool_name)
                    print(f"\n🔧 Tool Called: {tool_name}")
                    for key, value in tool_args.items():
                        display_value = str(value)[:80] + "..." if len(str(value)) > 80 else value
                        print(f"   └─ {key}: {display_value}")
        
        elif isinstance(msg, ToolMessage):
            result_preview = msg.content[:150] + "..." if len(msg.content) > 150 else msg.content
            print(f"\n📄 Result ({msg.name}): {result_preview}")
    
    # Show final answer
    print("\n" + "=" * 70)
    print("💬 FINAL ANSWER:")
    print("-" * 50)
    
    final_answer = None
    for msg in reversed(messages):
        if isinstance(msg, AIMessage) and msg.content and not msg.tool_calls:
            final_answer = msg.content
            break
    
    print(final_answer or "No answer generated.")
    
    # Highlight memory usage
    print("\n" + "=" * 70)
    memory_tools = [t for t in tool_calls_made if t in ["save_to_memory", "recall_memory"]]
    if memory_tools:
        print(f"🧠 MEMORY ACTIVITY: {memory_tools}")
    else:
        print("🧠 MEMORY ACTIVITY: None")
    print(f"📊 All tools used: {tool_calls_made}")
    print("=" * 70 + "\n")
    
    return run

#### Memory Demo: First Query (Learning)

This query requires web search. The agent should **save** what it learns to memory.

In [18]:
# First query - agent will likely need web search and should save to memory
# Using a question our database doesn't have
run_memory_1 = run_enhanced_query(
    "What is the release date of Grand Theft Auto 6?",
    session_id="memory_demo"
)

QUERY: What is the release date of Grand Theft Auto 6?
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

📋 AGENT REASONING & TOOL USAGE:
--------------------------------------------------

🔧 Tool Called: retrieve_game
   └─ query: Grand Theft Auto 6 release date

📄 Result (retrieve_game): "Game 1:\n  Name: Grand Theft Auto: San Andreas\n  Platform: PlayStation 2\n  Year of Release: 2004\n  Genre: Action-adventure\n  Publisher: Rockstar ...

🔧 Tool Called: recall_memory
   └─ query: Grand Theft Auto 6 release date

📄

#### Memory Demo: Second Query (Recalling)

Now we ask a related question. The agent should **recall** from memory instead of searching the web again!

In [19]:
# Second query - agent should recall from memory (no web search needed!)
# Reset the session so it doesn't have conversation context, only long-term memory
run_memory_2 = run_enhanced_query(
    "When is GTA 6 coming out?",
    session_id="memory_demo_2"  # New session - but memory persists!
)

QUERY: When is GTA 6 coming out?
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

📋 AGENT REASONING & TOOL USAGE:
--------------------------------------------------

🔧 Tool Called: game_web_search
   └─ question: GTA 6 release date

📄 Result (game_web_search): "Source 1:\n  Title: GTA 6 release date and time, delay, trailers, and latest news\n  URL: https://www.pcgamesn.com/grand-theft-auto-vi/gta-6-release-...

🔧 Tool Called: save_to_memory
   └─ information: GTA 6 is officially set to release on November 19, 2026.
   └─ topic: GTA 6 release date

📄 Result (save_to_memory): "Successfully saved to memory: 'GTA 6 release date'"

💬 FINAL ANSWER:
------------------------------------------------

### Final Summary

We've built a complete **UdaPlay AI Agent** with advanced features:

**Core Features:**
- `retrieve_game` - Vector database search
- `evaluate_retrieval` - LLM-as-judge quality assessment
- `game_web_search` - Web search fallback via Tavily

**Advanced Features:**
1. **Structured Output** - Returns both natural language AND JSON format
2. **Long-term Memory** - Agent learns from web searches and remembers for future queries
   - `save_to_memory` - Stores useful information
   - `recall_memory` - Retrieves past learnings

**Architecture:**
- State machine workflow for controlled execution
- Conversation state (short-term) via ShortTermMemory
- Persistent knowledge (long-term) via LongTermMemory + ChromaDB

The agent gets smarter over time as it learns from web searches! 🧠